**Implmentation for Baseline Logistic Regression and Penalised Regressions**

Section 1: Import Libraries and Setup

In [1]:
# Section 1: Import Libraries and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, precision_recall_curve, average_precision_score, f1_score
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel, RFE
import joblib
import os
import time
from datetime import datetime
import warnings
from sklearn.base import clone
from sklearn.utils import resample
from scipy import stats
warnings.filterwarnings('ignore')

# Create a directory for results if it doesn't exist
results_dir = f"results"
os.makedirs(results_dir, exist_ok=True)

# Set random seed for reproducibility
np.random.seed(42)


Section 2: Data Loading and Basic Information

In [2]:
# Section 2: Data Loading and Basic Information
# Load the data
print("Loading data...")
credit_data = pd.read_csv("UCI_Credit_Card.csv")

# Basic information about the dataset
print("\nBasic Information:")
print(f"Shape of dataset: {credit_data.shape}")
print("\nFirst few rows:")
print(credit_data.head())

# Check for missing values
print("\nMissing values in each column:")
print(credit_data.isnull().sum())

Loading data...

Basic Information:
Shape of dataset: (30000, 25)

First few rows:
   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1    20000.0    2          2         1   24      2      2     -1     -1   
1   2   120000.0    2          2         2   26     -1      2      0      0   
2   3    90000.0    2          2         2   34      0      0      0      0   
3   4    50000.0    2          2         1   37      0      0      0      0   
4   5    50000.0    1          2         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...        0.0        0.0        0.0       0.0     689.0       0.0   
1  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
2  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
3  ...    28314.0    28959.0    29547.0    2000.0    2019.0    1200.0   
4  ...    20940.0    19146.0    19131.0    2000.0   36681.0   10000.0   

   

Section 3: Data Preprocessing

In [3]:
# Section 3: Data Preprocessing
print("\n==== Data Preprocessing ====")

# Drop ID column (not useful for modeling)
credit_data = credit_data.drop('ID', axis=1)

# Check education and marriage for unusual values
print("\nUnique values in EDUCATION:", credit_data['EDUCATION'].unique())
print("Unique values in MARRIAGE:", credit_data['MARRIAGE'].unique())

# Fix education and marriage variables
# For education: 0, 5, 6 are mapped to 4 (Others)
# For marriage: 0 is mapped to 3 (Others)
credit_data['EDUCATION'] = credit_data['EDUCATION'].map(lambda x: 4 if x in [0, 5, 6] else x)
credit_data['MARRIAGE'] = credit_data['MARRIAGE'].map(lambda x: 3 if x == 0 else x)

print("\nAfter cleaning:")
print("Unique values in EDUCATION:", credit_data['EDUCATION'].unique())
print("Unique values in MARRIAGE:", credit_data['MARRIAGE'].unique())

# Feature and target variables
X = credit_data.drop('default.payment.next.month', axis=1)
y = credit_data['default.payment.next.month']

# Split data into training, validation and test sets (60%, 20%, 20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")

# Standardize numerical features
scaler = StandardScaler()
numerical_cols = ['LIMIT_BAL', 'AGE'] + [col for col in X.columns if col.startswith('BILL_') or col.startswith('PAY_AMT')]
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val[numerical_cols] = scaler.transform(X_val[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])


==== Data Preprocessing ====

Unique values in EDUCATION: [2 1 3 5 4 6 0]
Unique values in MARRIAGE: [1 2 3 0]

After cleaning:
Unique values in EDUCATION: [2 1 3 4]
Unique values in MARRIAGE: [1 2 3]
Training set shape: (18000, 23)
Validation set shape: (6000, 23)
Test set shape: (6000, 23)


Section 4: Model Evaluation Function Definition

In [7]:
# Section 4: Model Evaluation Function Definition
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name, selected_features=None):
    """Evaluate model performance with multiple metrics and visualizations"""
    start_time = time.time()
    
    # Use selected features if provided
    X_train_use = X_train[selected_features] if selected_features is not None else X_train
    X_val_use = X_val[selected_features] if selected_features is not None else X_val
    
    # Cross-validation on training data
    cv_scores = cross_val_score(model, X_train_use, y_train, cv=cv_strategy, scoring='roc_auc')
    
    # Fit the model on training data
    model.fit(X_train_use, y_train)
    
    # Predictions on validation data
    y_pred_proba = model.predict_proba(X_val_use)[:, 1]
    y_pred = model.predict(X_val_use)
    
    # Compute various metrics
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    avg_precision = average_precision_score(y_val, y_pred_proba)
    
    # Get ROC curve data
    fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
    
    # Classification report
    report = classification_report(y_val, y_pred)
    
    # Confusion matrix
    cm = confusion_matrix(y_val, y_pred)
    
    # Training time
    training_time = time.time() - start_time
    
    print(f"\n{model_name} Results:")
    print(f"Training time: {training_time:.2f} seconds")
    print(f"CV ROC-AUC Scores: {cv_scores}")
    print(f"Mean CV ROC-AUC: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    print(f"Validation ROC-AUC: {roc_auc:.4f}")
    print(f"Validation Avg Precision: {avg_precision:.4f}")
    
    print("\nClassification Report:")
    print(report)
    
    print("\nConfusion Matrix:")
    print(cm)
    
    # Plot individual ROC curve
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {model_name}')
    plt.legend()
    plt.savefig(os.path.join(results_dir, f'roc_curve_{model_name.replace(" ", "_").lower()}.png'))
    plt.close()
    
    # For models with coefficients, plot feature importance
    if hasattr(model, 'coef_'):
        features = selected_features if selected_features is not None else X_train.columns
        coef = model.coef_[0]
        
        # Sort coefficients and feature names
        indices = np.argsort(np.abs(coef))
        sorted_features = np.array(features)[indices]
        sorted_coef = coef[indices]
        
        plt.figure(figsize=(12, 10))
        plt.barh(range(len(sorted_coef)), sorted_coef)
        plt.yticks(range(len(sorted_coef)), sorted_features)
        plt.xlabel('Coefficient magnitude')
        plt.title(f'Feature Importance - {model_name}')
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, f'feature_importance_{model_name.replace(" ", "_").lower()}.png'))
        plt.close()
    
    # Return key metrics, model, and ROC curve data
    return {'avg_precision': avg_precision, 'roc_auc': roc_auc, 'model': model, 'roc_data': (fpr, tpr)}

# Dictionary to store models and their performance
models = {}

# Function to tune hyperparameters
def tune_hyperparameters(model, param_grid, X_train, y_train, cv=None, scoring='roc_auc'):
    """Tune hyperparameters using grid search and cross-validation"""
    if cv is None:
        cv = cv_strategy
        
    grid_search = GridSearchCV(
        model, param_grid, cv=cv, scoring=scoring, n_jobs=-1, verbose=1
    )
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV score: {grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_

Section 5: Baseline Model

In [8]:
# Section 5: Baseline Model
print("\n==== Model Building and Evaluation ====")

# Baseline Logistic Regression
print("\nTraining baseline Logistic Regression...")
baseline_model = LogisticRegression(random_state=42, max_iter=2000)
models = {}
models['Baseline Logistic Regression'] = evaluate_model(baseline_model, X_train, y_train, X_val, y_val, "Baseline Logistic Regression")


==== Model Building and Evaluation ====

Training baseline Logistic Regression...

Baseline Logistic Regression Results:
Training time: 0.59 seconds
CV ROC-AUC Scores: [0.74433195 0.72314923 0.71371748 0.71553735 0.7286708 ]
Mean CV ROC-AUC: 0.7251 (±0.0110)
Validation ROC-AUC: 0.7030
Validation Avg Precision: 0.4785

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.97      0.88      4673
           1       0.66      0.23      0.34      1327

    accuracy                           0.80      6000
   macro avg       0.74      0.60      0.61      6000
weighted avg       0.78      0.80      0.76      6000


Confusion Matrix:
[[4519  154]
 [1027  300]]


Section 6: Ridge Regression

In [9]:
# Section 6: Ridge Regression
# Ridge Logistic Regression (L2 regularization) with hyperparameter tuning
print("\nTraining Ridge Logistic Regression with Hyperparameter Tuning...")
ridge_param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'class_weight': [None, 'balanced', {0: 1, 1: 3}, {0: 1, 1: 4}]
}
ridge_base = LogisticRegression(penalty='l2', random_state=42, max_iter=2000)
ridge_tuned = tune_hyperparameters(ridge_base, ridge_param_grid, X_train, y_train)
models['Ridge Logistic Regression (Tuned)'] = evaluate_model(ridge_tuned, X_train, y_train, X_val, y_val, "Ridge Logistic Regression (Tuned)")


Training Ridge Logistic Regression with Hyperparameter Tuning...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters: {'C': 1.0, 'class_weight': {0: 1, 1: 4}}
Best CV score: 0.7256

Ridge Logistic Regression (Tuned) Results:
Training time: 1.06 seconds
CV ROC-AUC Scores: [0.74539244 0.72341177 0.71468433 0.71497468 0.72933821]
Mean CV ROC-AUC: 0.7256 (±0.0113)
Validation ROC-AUC: 0.7034
Validation Avg Precision: 0.4757

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.60      0.71      4673
           1       0.32      0.67      0.44      1327

    accuracy                           0.62      6000
   macro avg       0.59      0.64      0.57      6000
weighted avg       0.75      0.62      0.65      6000


Confusion Matrix:
[[2815 1858]
 [ 437  890]]


Section 7: Lasso Regression

In [10]:
# Section 7: Lasso Regression
# Lasso Logistic Regression (L1 regularization) with hyperparameter tuning
print("\nTraining Lasso Logistic Regression with Hyperparameter Tuning...")
lasso_param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],
    'class_weight': [None, 'balanced', {0: 1, 1: 3}, {0: 1, 1: 4}]
}
lasso_base = LogisticRegression(penalty='l1', solver='liblinear', random_state=42, max_iter=2000)
lasso_tuned = tune_hyperparameters(lasso_base, lasso_param_grid, X_train, y_train)
models['Lasso Logistic Regression (Tuned)'] = evaluate_model(lasso_tuned, X_train, y_train, X_val, y_val, "Lasso Logistic Regression (Tuned)")


Training Lasso Logistic Regression with Hyperparameter Tuning...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters: {'C': 1.0, 'class_weight': {0: 1, 1: 4}}
Best CV score: 0.7256

Lasso Logistic Regression (Tuned) Results:
Training time: 1.69 seconds
CV ROC-AUC Scores: [0.74537721 0.72345344 0.71462385 0.71501094 0.72941744]
Mean CV ROC-AUC: 0.7256 (±0.0113)
Validation ROC-AUC: 0.7034
Validation Avg Precision: 0.4757

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.60      0.71      4673
           1       0.32      0.67      0.44      1327

    accuracy                           0.62      6000
   macro avg       0.60      0.64      0.57      6000
weighted avg       0.75      0.62      0.65      6000


Confusion Matrix:
[[2815 1858]
 [ 436  891]]


Section 8: Elastic Net

In [11]:
# Section 8: Elastic Net
# Elastic Net Logistic Regression with hyperparameter tuning
print("\nTraining Elastic Net Logistic Regression with Hyperparameter Tuning...")
elastic_net_param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0],
    'l1_ratio': [0.2, 0.5, 0.8],
    'class_weight': [None, 'balanced', {0: 1, 1: 3}, {0: 1, 1: 4}]
}
elastic_net_base = LogisticRegression(penalty='elasticnet', solver='saga', random_state=42, max_iter=2000)
elastic_net_tuned = tune_hyperparameters(elastic_net_base, elastic_net_param_grid, X_train, y_train)
models['Elastic Net Logistic Regression (Tuned)'] = evaluate_model(elastic_net_tuned, X_train, y_train, X_val, y_val, "Elastic Net Logistic Regression (Tuned)")


Training Elastic Net Logistic Regression with Hyperparameter Tuning...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'C': 1.0, 'class_weight': {0: 1, 1: 4}, 'l1_ratio': 0.8}
Best CV score: 0.7256

Elastic Net Logistic Regression (Tuned) Results:
Training time: 5.05 seconds
CV ROC-AUC Scores: [0.74538572 0.72344448 0.71462385 0.71499079 0.72940625]
Mean CV ROC-AUC: 0.7256 (±0.0113)
Validation ROC-AUC: 0.7034
Validation Avg Precision: 0.4757

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.60      0.71      4673
           1       0.32      0.67      0.44      1327

    accuracy                           0.62      6000
   macro avg       0.59      0.64      0.57      6000
weighted avg       0.75      0.62      0.65      6000


Confusion Matrix:
[[2812 1861]
 [ 436  891]]


Section 9: Forward Selection

In [12]:
# Section 9: Forward Selection
# Forward Selection with different class weights
print("\nTraining Forward Selection with different class weights...")
class_weights = [None, 'balanced', {0: 1, 1: 3}, {0: 1, 1: 4}]
best_forward_auc = 0
best_forward_model = None
best_forward_weight = None

for weight in class_weights:
    print(f"\nTrying Forward Selection with class_weight: {weight}")
    forward_selector = SequentialFeatureSelector(
        LogisticRegression(random_state=42, max_iter=2000, class_weight=weight),
        n_features_to_select=10,
        direction='forward',
        scoring='roc_auc',
        cv=3
    )
    forward_selector.fit(X_train, y_train)
    forward_features = X_train.columns[forward_selector.get_support()]
    print(f"Selected features (Forward): {forward_features.tolist()}")
    
    # Train model with selected features
    forward_model = LogisticRegression(random_state=42, max_iter=2000, class_weight=weight)
    results = evaluate_model(forward_model, X_train, y_train, X_val, y_val, f"Forward Selection (weight={weight})", forward_features)
    
    if results['roc_auc'] > best_forward_auc:
        best_forward_auc = results['roc_auc']
        best_forward_model = forward_model
        best_forward_weight = weight
        best_forward_features = forward_features

models['Forward Selection'] = {'model': best_forward_model, 'roc_auc': best_forward_auc, 'features': best_forward_features}
print(f"\nBest Forward Selection class weight: {best_forward_weight} with ROC-AUC: {best_forward_auc:.4f}")


Training Forward Selection with different class weights...

Trying Forward Selection with class_weight: None
Selected features (Forward): ['LIMIT_BAL', 'SEX', 'PAY_0', 'PAY_3', 'PAY_4', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

Forward Selection (weight=None) Results:
Training time: 0.24 seconds
CV ROC-AUC Scores: [0.74172888 0.72705293 0.71696639 0.7183234  0.73335322]
Mean CV ROC-AUC: 0.7275 (±0.0093)
Validation ROC-AUC: 0.7071
Validation Avg Precision: 0.4791

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.97      0.89      4673
           1       0.68      0.23      0.34      1327

    accuracy                           0.81      6000
   macro avg       0.75      0.60      0.61      6000
weighted avg       0.79      0.81      0.77      6000


Confusion Matrix:
[[4532  141]
 [1024  303]]

Trying Forward Selection with class_weight: balanced
Selected features (Forward): ['LIMIT_BAL', 'SEX', 'PAY_0', 'PA

Section 10: Backward Selection

In [13]:
# Section 10: Backward Selection
# Backward Selection with different class weights
print("\nTraining Backward Selection with different class weights...")
class_weights = [None, 'balanced', {0: 1, 1: 3}, {0: 1, 1: 4}]
best_backward_auc = 0
best_backward_model = None
best_backward_weight = None

for weight in class_weights:
    print(f"\nTrying Backward Selection with class_weight: {weight}")
    
    # Initialize variables to store models and their metrics
    models_by_k = {}
    features_remaining = X_train.columns.tolist()
    n_features = len(features_remaining)
    
    # For each k from p down to 1
    for k in range(n_features, 0, -1):
        # Fit model with current features
        model = LogisticRegression(random_state=42, max_iter=2000, class_weight=weight)
        model.fit(X_train[features_remaining], y_train)
        
        # Calculate t-statistics for each feature
        y_pred = model.predict(X_train[features_remaining])
        residuals = y_train - y_pred
        
        # Calculate standard errors using bootstrap
        n_bootstrap = 100
        coef_samples = np.zeros((n_bootstrap, len(features_remaining)))
        
        for i in range(n_bootstrap):
            # Bootstrap sample indices
            indices = np.random.choice(len(y_train), len(y_train), replace=True)
            # Fit model on bootstrap sample
            model_boot = LogisticRegression(random_state=42, max_iter=2000, class_weight=weight)
            model_boot.fit(X_train[features_remaining].iloc[indices], y_train.iloc[indices])
            coef_samples[i, :] = model_boot.coef_[0]
        
        # Calculate standard errors
        std_errors = np.std(coef_samples, axis=0)
        t_stats = np.abs(model.coef_[0] / std_errors)
        
        # Store current model and its cross-validated metrics
        cv_scores = cross_val_score(model, X_train[features_remaining], y_train, 
                                  cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                                  scoring='roc_auc')
        
        models_by_k[k] = {
            'features': features_remaining.copy(),
            'model': clone(model),
            'cv_roc_auc_mean': cv_scores.mean(),
            'cv_roc_auc_std': cv_scores.std()
        }
        
        # If we need to remove more features
        if k > 1:
            # Find feature with smallest absolute t-statistic
            min_t_idx = np.argmin(t_stats)
            # Remove that feature
            removed_feature = features_remaining.pop(min_t_idx)
            print(f"Dropped feature '{removed_feature}' (t-stat: {t_stats[min_t_idx]:.4f})")
    
    # Select best model using cross-validated ROC-AUC
    best_k = max(models_by_k.keys(), key=lambda k: models_by_k[k]['cv_roc_auc_mean'])
    best_model = models_by_k[best_k]['model']
    selected_features = models_by_k[best_k]['features']
    
    print(f"\nBest model for this weight has {best_k} features:")
    print(f"Selected features: {selected_features}")
    print(f"Mean CV ROC-AUC: {models_by_k[best_k]['cv_roc_auc_mean']:.4f} (±{models_by_k[best_k]['cv_roc_auc_std']:.4f})")
    
    # Evaluate on validation set
    results = evaluate_model(best_model, X_train, y_train, X_val, y_val, 
                           f"Backward Selection (weight={weight})", selected_features)
    
    if results['roc_auc'] > best_backward_auc:
        best_backward_auc = results['roc_auc']
        best_backward_model = best_model
        best_backward_weight = weight
        best_backward_features = selected_features

models['Backward Selection'] = {'model': best_backward_model, 'roc_auc': best_backward_auc, 'features': best_backward_features}
print(f"\nBest Backward Selection class weight: {best_backward_weight} with ROC-AUC: {best_backward_auc:.4f}")



Training Backward Selection with different class weights...

Trying Backward Selection with class_weight: None
Dropped feature 'BILL_AMT6' (t-stat: 0.0019)
Dropped feature 'BILL_AMT2' (t-stat: 0.0397)
Dropped feature 'PAY_AMT5' (t-stat: 0.1437)
Dropped feature 'BILL_AMT5' (t-stat: 0.2366)
Dropped feature 'PAY_5' (t-stat: 0.3839)
Dropped feature 'PAY_AMT3' (t-stat: 0.6287)
Dropped feature 'BILL_AMT4' (t-stat: 0.2367)
Dropped feature 'PAY_AMT4' (t-stat: 0.9549)
Dropped feature 'PAY_AMT6' (t-stat: 1.1021)
Dropped feature 'PAY_6' (t-stat: 1.5420)
Dropped feature 'BILL_AMT3' (t-stat: 1.9808)
Dropped feature 'PAY_AMT1' (t-stat: 2.0973)
Dropped feature 'PAY_3' (t-stat: 1.8506)
Dropped feature 'AGE' (t-stat: 2.3994)
Dropped feature 'SEX' (t-stat: 3.1665)
Dropped feature 'PAY_AMT2' (t-stat: 2.6110)
Dropped feature 'EDUCATION' (t-stat: 3.7277)
Dropped feature 'PAY_2' (t-stat: 4.7961)
Dropped feature 'MARRIAGE' (t-stat: 4.9659)
Dropped feature 'BILL_AMT1' (t-stat: 5.5906)
Dropped feature 'PAY_4'

Section 11: Model Comparison and Visualization

In [14]:
# Section 11: Model Comparison and Visualization
print("\n==== Creating Combined ROC Curve Plot ====")
plt.figure(figsize=(12, 10))
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']

# Store best models and their metrics for comparison
best_models = {
    'Baseline LR': baseline_model,
    'Ridge (w={0: 1, 1: 4})': ridge_tuned,
    'Lasso (w={0: 1, 1: 4})': lasso_tuned,
    'Elastic Net (w={0: 1, 1: 4})': elastic_net_tuned,
    'Forward Selection (no weights)': best_forward_model,
    'Backward Selection (w={0: 1, 1: 3})': best_backward_model
}

# Plot ROC curves for best models
for (model_name, model), color in zip(best_models.items(), colors):
    if model_name.startswith('Forward Selection'):
        y_pred_proba = model.predict_proba(X_val[best_forward_features])[:, 1]
    elif model_name.startswith('Backward Selection'):
        y_pred_proba = model.predict_proba(X_val[best_backward_features])[:, 1]
    else:
        y_pred_proba = model.predict_proba(X_val)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    plt.plot(fpr, tpr, color=color, label=f'{model_name} (AUC = {roc_auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Combined ROC Curves for Best Models')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'combined_roc_curves_best_models.png'), bbox_inches='tight', dpi=300)
plt.close()


# Create comparison chart for best models
print("\n==== Model Comparison for Best Models ====")
model_metrics = []

for model_name, model in best_models.items():
    if model_name.startswith('Forward Selection'):
        y_pred_proba = model.predict_proba(X_val[best_forward_features])[:, 1]
    elif model_name.startswith('Backward Selection'):
        y_pred_proba = model.predict_proba(X_val[best_backward_features])[:, 1]
    else:
        y_pred_proba = model.predict_proba(X_val)[:, 1]
    
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    avg_precision = average_precision_score(y_val, y_pred_proba)
    model_metrics.append({
        'Model': model_name,
        'ROC-AUC': roc_auc,
        'Avg Precision': avg_precision
    })

# Sort models by ROC-AUC performance
model_metrics = sorted(model_metrics, key=lambda x: x['ROC-AUC'], reverse=True)

# Print metrics
print("\nModels ranked by ROC-AUC performance:")
for i, metrics in enumerate(model_metrics, 1):
    print(f"{i}. {metrics['Model']}: ROC-AUC = {metrics['ROC-AUC']:.4f}, Avg Precision = {metrics['Avg Precision']:.4f}")

# Create bar plot comparing models
plt.figure(figsize=(14, 8))
models_names = [m['Model'] for m in model_metrics]
roc_auc_scores = [m['ROC-AUC'] for m in model_metrics]

bars = plt.bar(models_names, roc_auc_scores)
plt.title('Model Comparison - ROC-AUC Scores (Best Models)')
plt.xlabel('Models')
plt.ylabel('ROC-AUC Score')
plt.ylim(0.702, 0.708)  # Adjusted y-axis limits to zoom in on the differences
plt.xticks(rotation=45, ha='right')
bars[0].set_color('red')  # Highlight the best model
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Added more visible grid
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'model_comparison_best_models.png'))
plt.close()

# Find the overall best model
best_model_metrics = max(model_metrics, key=lambda x: x['ROC-AUC'])
best_model_name = best_model_metrics['Model']
best_auc = best_model_metrics['ROC-AUC']

print(f"\n==== Best Model: {best_model_name} with ROC-AUC {best_auc:.4f} ====")

def find_optimal_threshold(y_true, y_pred_proba):
    """Find the probability threshold that maximizes F1 score"""
    thresholds = np.arange(0.1, 1.0, 0.01)
    f1_scores = []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred)
        f1_scores.append(f1)
    
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    max_f1 = max(f1_scores)
    
    # Plot F1 scores vs thresholds
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, f1_scores)
    plt.axvline(x=optimal_threshold, color='r', linestyle='--', 
                label=f'Optimal threshold = {optimal_threshold:.2f}')
    plt.xlabel('Threshold')
    plt.ylabel('F1 Score')
    plt.title('F1 Score vs Classification Threshold')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(results_dir, 'f1_vs_threshold.png'))
    plt.close()
    
    return optimal_threshold, max_f1



==== Creating Combined ROC Curve Plot ====

==== Model Comparison for Best Models ====

Models ranked by ROC-AUC performance:
1. Forward Selection (no weights): ROC-AUC = 0.7071, Avg Precision = 0.4791
2. Lasso (w={0: 1, 1: 4}): ROC-AUC = 0.7034, Avg Precision = 0.4757
3. Elastic Net (w={0: 1, 1: 4}): ROC-AUC = 0.7034, Avg Precision = 0.4757
4. Backward Selection (w={0: 1, 1: 3}): ROC-AUC = 0.7034, Avg Precision = 0.4752
5. Ridge (w={0: 1, 1: 4}): ROC-AUC = 0.7034, Avg Precision = 0.4757
6. Baseline LR: ROC-AUC = 0.7030, Avg Precision = 0.4785

==== Best Model: Forward Selection (no weights) with ROC-AUC 0.7071 ====


Section 12: Final Model Selection and Evaluation

In [15]:
# Section 12: Final Model Selection and Evaluation
print("\n==== Final Evaluation on Test Set ====")
# Get the best model and its features
best_model = best_models[best_model_name]
selected_features = None
if best_model_name.startswith('Forward Selection'):
    selected_features = best_forward_features
elif best_model_name.startswith('Backward Selection'):
    selected_features = best_backward_features

# For the best model, evaluate on both validation and test sets
if selected_features is not None:
    print(f"\nUsing {len(selected_features)} selected features for final evaluation")
    X_val_use = X_val[selected_features]
    X_test_use = X_test[selected_features]
else:
    X_val_use = X_val
    X_test_use = X_test

# Fit the model on training data
best_model.fit(X_train if selected_features is None else X_train[selected_features], y_train)

# Get predictions on validation set
val_pred_proba = best_model.predict_proba(X_val_use)[:, 1]
val_roc_auc = roc_auc_score(y_val, val_pred_proba)

# Find optimal threshold using validation set
print("\nFinding optimal threshold using validation set...")
optimal_threshold, val_max_f1 = find_optimal_threshold(y_val, val_pred_proba)
print(f"Optimal threshold: {optimal_threshold:.3f}")
print(f"Validation ROC-AUC: {val_roc_auc:.3f}")
print(f"Validation F1-score at optimal threshold: {val_max_f1:.3f}")

# Apply to test set
test_pred_proba = best_model.predict_proba(X_test_use)[:, 1]
test_pred = (test_pred_proba >= optimal_threshold).astype(int)
test_roc_auc = roc_auc_score(y_test, test_pred_proba)
test_f1 = f1_score(y_test, test_pred)

# Save test set predictions and actual values
test_results = pd.DataFrame({
    'actual': y_test,
    'predicted': test_pred,
    'probability': test_pred_proba
})
test_results.to_csv(os.path.join(results_dir, 'logistic_regression_test_predictions.csv'), index=False)

print("\nTest Set Performance:")
print(f"Test ROC-AUC: {test_roc_auc:.3f}")
print(f"Test F1-score at optimal threshold: {test_f1:.3f}")

print("\nClassification Report at Optimal Threshold:")
print(classification_report(y_test, test_pred))

# Save the best model and optimal threshold
joblib.dump({
    'model': best_model,
    'optimal_threshold': optimal_threshold,
    'selected_features': selected_features
}, os.path.join(results_dir, 'best_model.pkl'))

print(f"\nAnalysis complete. Results and visualizations have been saved to the '{results_dir}' directory.")
print(f"Test set predictions have been saved to '{results_dir}/logistic_regression_test_predictions.csv'") 


==== Final Evaluation on Test Set ====

Using 10 selected features for final evaluation

Finding optimal threshold using validation set...
Optimal threshold: 0.250
Validation ROC-AUC: 0.707
Validation F1-score at optimal threshold: 0.502

Test Set Performance:
Test ROC-AUC: 0.727
Test F1-score at optimal threshold: 0.525

Classification Report at Optimal Threshold:
              precision    recall  f1-score   support

           0       0.87      0.84      0.86      4673
           1       0.50      0.55      0.53      1327

    accuracy                           0.78      6000
   macro avg       0.68      0.70      0.69      6000
weighted avg       0.79      0.78      0.78      6000


Analysis complete. Results and visualizations have been saved to the 'results' directory.
Test set predictions have been saved to 'results/logistic_regression_test_predictions.csv'
